<a href="https://colab.research.google.com/github/MrRox1337/CST3170/blob/main/Coursework%202/coursework2Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CST3170 Coursework - AI Model Development

By: Aman Mishra
MISIS: M00983641
Professor: Dr. Maha Saadeh

This notebook uses training dataset to learn the type of digit based on neural activations. It generates the prediction model and applies it to a new testing dataset to predict a base10 digit.

The training uses Neural Netwoks, especially Multi-Layered Perceptrons to create hidden layers and apply feed forward functions to get a bias toward the correct number.

## Setup

In this section we will set up the dataset repository, neurons, and functions used for Multi-Layered Perceptron.

### Import datasets from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Import modules for matrix manipulation

In [ ]:
import numpy as np
import pandas as pd

### Hidden layers and weights

In [ ]:
# Set h (number of hidden neurons)
h = 64  # Number of hidden neurons (can be modified)

# Global arrays for outputs
outputHidden = np.zeros(h)  # Hidden layer outputs
outputneuron = np.zeros(10)  # Output layer outputs

# Initialize random weights for the input-hidden and hidden-output connections
WH = np.random.uniform(-1, 1, (h, 64))  # Weights between input and hidden layer
WO = np.random.uniform(-1, 1, (10, h))  # Weights between hidden and output layer

### Feedfoward function

In [ ]:
# Feedforward method to calculate outputs
def feedforward(dataSample):
    global outputHidden, outputneuron

    # Compute the output of the hidden neurons
    for i in range(h):
        weighted_sum_hidden = np.dot(dataSample, WH[i])  # Weighted sum for hidden neuron i
        outputHidden[i] = sigmoid(weighted_sum_hidden)  # Apply sigmoid

    # Compute the output of the output neurons
    for i in range(10):
        weighted_sum_output = np.dot(outputHidden, WO[i])  # Weighted sum for output neuron i
        outputneuron[i] = 1 if weighted_sum_output >= 0 else 0  # Apply threshold

### Sigmoid function

In [ ]:
# Sigmoid activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

### Validation function

In [ ]:
# Test error method to compare outputneuron with Map
def testError(Map):
    for i in range(10):
        if Map[i] != outputneuron[i]:
            return True
    return False

### Model training function

In [ ]:
# Training method to adjust weights based on the errors
def training(Map, dataSample, learningRate=0.0125):
    global WH, WO

    # Compute error for the output neurons
    errorOutput = np.zeros(10)  # Error for the output neurons
    for i in range(10):
        errorOutput[i] = Map[i] - outputneuron[i]

    # Compute error for the hidden neurons
    errorHidden = np.zeros(h)  # Error for the hidden neurons
    for j in range(h):
        errorTemp = 0
        for i in range(10):
            errorTemp += errorOutput[i] * WO[i][j]
        errorHidden[j] = outputHidden[j] * (1 - outputHidden[j]) * errorTemp

    # Adjust weights between hidden and output neurons (WO)
    for i in range(10):  # Loop over output neurons
        for j in range(h):  # Loop over hidden neurons
            WO[i][j] += learningRate * outputHidden[j] * errorOutput[i]

    # Adjust weights between input and hidden neurons (WH)
    for i in range(h):  # Loop over hidden neurons
        for j in range(64):  # Loop over input data sample
            WH[i][j] += learningRate * dataSample[j] * errorHidden[i]

## Training

### Load training dataset and set hyperparameters

In [ ]:
# Load the training data from CSV
training_data_path = '/content/drive/MyDrive/cw2DataSet1.csv'  # Ensure this is the correct path
training_data = pd.read_csv(training_data_path)

# Set learning rate and number of cycles
learningRate = 0.0095
total_cycles = 550  # Total number of training cycles
display_interval = 50  # Display accuracy after every 50 cycles

### Build prediction model

In [ ]:
# Iterate through the cycles
for cycle in range(total_cycles):
    success = 0
    total_rows = len(training_data)

    # Process each data sample
    for _, row in training_data.iterrows():
        dataSample = row[:-1].values  # First 64 columns as dataSample
        targetOutput = int(row.iloc[-1])  # Last column as targetOutput using iloc

        # Initialize the Map array to zeros
        Map = np.zeros(10)
        Map[targetOutput] = 1  # Set the target output index to 1

        # Feedforward
        feedforward(dataSample)

        # Test error and update weights if needed
        if testError(Map):
            training(Map, dataSample, learningRate)
        else:
            success += 1

    # Calculate accuracy at the end of each cycle
    accuracy = success / total_rows

    # Display the accuracy after every 50 cycles
    if ((cycle + 1) % display_interval == 0) or (cycle <= 20):
        print(f"Cycle {cycle + 1}, Accuracy: {accuracy:.4f}")

Cycle 1, Accuracy: 0.3293
Cycle 2, Accuracy: 0.5685
Cycle 3, Accuracy: 0.6614
Cycle 4, Accuracy: 0.7191
Cycle 5, Accuracy: 0.7433
Cycle 6, Accuracy: 0.7811
Cycle 7, Accuracy: 0.7917
Cycle 8, Accuracy: 0.8156
Cycle 9, Accuracy: 0.8309
Cycle 10, Accuracy: 0.8423
Cycle 11, Accuracy: 0.8451
Cycle 12, Accuracy: 0.8537
Cycle 13, Accuracy: 0.8601
Cycle 14, Accuracy: 0.8565
Cycle 15, Accuracy: 0.8683
Cycle 16, Accuracy: 0.8815
Cycle 17, Accuracy: 0.8825
Cycle 18, Accuracy: 0.8889
Cycle 19, Accuracy: 0.8978
Cycle 20, Accuracy: 0.8925
Cycle 21, Accuracy: 0.8996
Cycle 50, Accuracy: 0.9534
Cycle 100, Accuracy: 0.9754
Cycle 150, Accuracy: 0.9811
Cycle 200, Accuracy: 0.9868
Cycle 250, Accuracy: 0.9907
Cycle 300, Accuracy: 0.9900
Cycle 350, Accuracy: 0.9872
Cycle 400, Accuracy: 0.9900
Cycle 450, Accuracy: 0.9979
Cycle 500, Accuracy: 0.9986
Cycle 550, Accuracy: 1.0000


## Testing

### Load test dataset

In [ ]:
# Load the testing data from CSV
testing_data_path = '/content/drive/MyDrive/cw2DataSet2.csv'  # Ensure this is the correct path
testing_data = pd.read_csv(training_data_path)

### Test generated model

In [ ]:
success = 0
total_rows = len(testing_data)

# Process each data sample
for _, row in testing_data.iterrows():
    dataSample = row[:-1].values  # First 64 columns as dataSample
    targetOutput = int(row.iloc[-1])  # Last column as targetOutput using iloc

    # Initialize the Map array to zeros
    Map = np.zeros(10)
    Map[targetOutput] = 1  # Set the target output index to 1

    # Feedforward
    feedforward(dataSample)

    # Test error and update weights if needed
    if not testError(Map):
        success += 1

# Calculate accuracy at the end of testing
accuracy = success / total_rows
print(f"Testing Accuracy: {accuracy:.4f}")

Testing Accuracy: 1.0000
